In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Dropout, BatchNormalization, Add, Average, Concatenate, Lambda)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
import keras_tuner as kt
import os

In [3]:
data = pd.read_csv("Brain.tsv", sep='\t')
features = ['tiv', 'csfv', 'gmv', 'wmv']
target = 'age'
X = data[features].values
y = data[target].values

print("Features:")
print(data[features].head())

print("\nTarget:")
print(data[target].head())

Features:
           tiv        csfv         gmv         wmv
0  1434.357361  219.565569  678.230161  535.878459
1  1558.161428  261.968669  756.742506  538.817738
2  1418.050690  242.123816  686.814910  488.740580
3  1304.233543  206.305238  667.809720  429.723510
4  1660.856147  344.783456  703.484560  611.222413

Target:
0    19.0
1    21.0
2    21.0
3    15.0
4    31.0
Name: age, dtype: float64


In [17]:
data.describe()

,participant_id,age,tiv,csfv,gmv,wmv,magnetic_field_strength,acquisition_setting,site
count,3.984000e+03,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000
mean,5.542203e+11,24.922390,1450.235934,253.048198,685.158274,511.076201,2.869352,1.207078,17.887299
std,2.605308e+11,14.287559,144.051540,62.063662,88.062662,62.508482,0.423022,0.494550,16.739054
min,1.000532e+11,5.900000,762.709988,100.219412,260.969929,313.087018,1.500000,1.000000,0.000000
25%,3.299751e+11,19.000000,1348.497432,211.831925,629.440521,466.820755,3.000000,1.000000,3.000000
50%,5.485667e+11,21.000000,1444.412188,243.370296,684.537534,507.093032,3.000000,1.000000,14.000000
75%,7.823813e+11,26.000000,1548.161334,283.595180,742.164149,552.503305,3.000000,1.000000,25.000000
max,9.998326e+11,88.000000,2101.970447,624.906527,986.262878,808.156251,3.000000,3.000000,63.000000


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

import joblib

# After fitting the scaler:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler to a file called "scaler.save"
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [7]:
#####################################
# Model 1: Ultimate Super-Residual Model (Ultra_ResDNN)
#####################################
def build_super_residual_model(hp):
    inputs = Input(shape=(X_train_scaled.shape[1],))
    
    # Base parameters
    base_units = hp.Int('base_units', min_value=128, max_value=512, step=64)
    reg = regularizers.l2(hp.Float('l2_reg', 1e-5, 1e-3, sampling='LOG'))
    dropout_rate = hp.Float('dropout_rate', 0.2, 0.5, step=0.1)
    
    # Initial layer
    x = Dense(units=base_units, activation='relu', kernel_regularizer=reg)(inputs)
    x = BatchNormalization()(x)
    
    # Multiple residual blocks
    num_blocks = hp.Int('num_blocks', min_value=3, max_value=6, step=1)
    for i in range(num_blocks):
        shortcut = x
        x = Dense(units=base_units, activation='relu', kernel_regularizer=reg)(x)
        x = BatchNormalization()(x)
        x = Dropout(rate=dropout_rate)(x)
        x = Dense(units=base_units, activation='relu', kernel_regularizer=reg)(x)
        x = BatchNormalization()(x)
        # Residual connection (dimensions match)
        x = Add()([shortcut, x])
    
    # Final dense layers after residual blocks
    post_units = hp.Int('post_units', min_value=64, max_value=256, step=32)
    x = Dense(units=post_units, activation='relu', kernel_regularizer=reg)(x)
    x = Dropout(rate=hp.Float('dropout_post', 0.1, 0.4, step=0.1))(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs, name="Ultra_ResDNN")
    model.compile(optimizer=Adam(learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='LOG')),
                  loss='mae', metrics=['mse','mae'])
    return model

tuner_residual = kt.RandomSearch(
    build_super_residual_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=2,
    directory='tuner_super_residual',
    project_name='brain_age_ultra_residual',
    overwrite=True
)

tuner_residual.search(X_train_scaled, y_train, validation_split=0.2, epochs=500,
                        callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
                        verbose=1)
best_residual_model = tuner_residual.get_best_models(num_models=1)[0]

Trial 50 Complete [00h 08m 03s]
val_loss: 4.436497449874878

Best val_loss So Far: 4.365634441375732
Total elapsed time: 09h 58m 20s


d:\Brain Age\brain_age_env\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 66 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
#####################################
# Model 2: Extreme Wide & Deep Neural Network (Ultra_WideDeep)
#####################################
def build_extreme_wide_deep_model(hp):
    inputs = Input(shape=(X_train_scaled.shape[1],))
    reg = regularizers.l2(hp.Float('l2_reg', 1e-5, 1e-3, sampling='LOG'))
    
    # Deep branch: many layers
    deep = Dense(units=hp.Int('deep_units1', min_value=256, max_value=1024, step=128), activation='relu', kernel_regularizer=reg)(inputs)
    deep = BatchNormalization()(deep)
    deep = Dropout(rate=hp.Float('dropout_deep1', 0.2, 0.6, step=0.1))(deep)
    deep = Dense(units=hp.Int('deep_units2', min_value=128, max_value=512, step=64), activation='relu', kernel_regularizer=reg)(deep)
    deep = BatchNormalization()(deep)
    deep = Dropout(rate=hp.Float('dropout_deep2', 0.2, 0.6, step=0.1))(deep)
    deep = Dense(units=hp.Int('deep_units3', min_value=64, max_value=256, step=32), activation='relu', kernel_regularizer=reg)(deep)
    
    # Wide branch: direct connection (linear)
    wide = Dense(1, activation='linear')(inputs)
    
    # Combine deep and wide
    combined = Concatenate()([deep, wide])
    combined = Dense(units=hp.Int('combined_units', min_value=128, max_value=512, step=64), activation='relu', kernel_regularizer=reg)(combined)
    combined = Dropout(rate=hp.Float('dropout_combined', 0.1, 0.5, step=0.1))(combined)
    outputs = Dense(1)(combined)
    
    model = Model(inputs, outputs, name="Ultra_WideDeep")
    model.compile(optimizer=Adam(learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='LOG')),
                  loss='mae', metrics=['mse','mae'])
    return model

tuner_wide_deep = kt.RandomSearch(
    build_extreme_wide_deep_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=2,
    directory='tuner_extreme_wide_deep',
    project_name='brain_age_ultra_wide_deep',
    overwrite=True
)

tuner_wide_deep.search(X_train_scaled, y_train, validation_split=0.2, epochs=500,
                         callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
                         verbose=1)
best_wide_deep_model = tuner_wide_deep.get_best_models(num_models=1)[0]

Trial 50 Complete [00h 07m 24s]
val_loss: 4.4709203243255615

Best val_loss So Far: 4.293081998825073
Total elapsed time: 05h 41m 35s


d:\Brain Age\brain_age_env\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
#####################################
# Model 3: Cutting-Edge Attention Neural Network (Ultra_Attention)
#####################################
from tensorflow.keras.layers import MultiHeadAttention, Reshape
from tensorflow.keras import regularizers

def build_advanced_attention_model(hp):
    inputs = Input(shape=(X_train_scaled.shape[1],))
    reg = regularizers.l2(hp.Float('l2_reg', 1e-5, 1e-3, sampling='LOG'))
    
    # Determine projection dimension and compute total units needed for reshaping.
    proj_dim = hp.Int('proj_dim', min_value=32, max_value=128, step=16)
    num_features = X_train_scaled.shape[1]  # should be 4 in this case
    total_units = num_features * proj_dim  # must match the total elements in the target shape

    # Project inputs to a vector of length total_units.
    x_proj = Dense(total_units, activation='relu', kernel_regularizer=reg)(inputs)
    
    # Reshape to (batch_size, num_features, proj_dim)
    x_reshaped = Reshape((num_features, proj_dim))(x_proj)
    
    # Multi-Head Attention layer
    num_heads = hp.Int('num_heads', min_value=2, max_value=4, step=1)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=proj_dim//num_heads)(x_reshaped, x_reshaped)
    
    # Flatten the output from multi-head attention
    attn_flat = tf.keras.layers.Flatten()(attn_output)
    
    # Deep layers after attention
    x = Dense(units=hp.Int('dense_units1', min_value=256, max_value=1024, step=128),
              activation='relu', kernel_regularizer=reg)(attn_flat)
    x = BatchNormalization()(x)
    x = Dropout(rate=hp.Float('dropout_att1', 0.2, 0.6, step=0.1))(x)
    x = Dense(units=hp.Int('dense_units2', min_value=128, max_value=512, step=64),
              activation='relu', kernel_regularizer=reg)(x)
    x = BatchNormalization()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs, name="Ultra_Attention")
    model.compile(optimizer=Adam(learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='LOG')),
                  loss='mae', metrics=['mse','mae'])
    return model

tuner_attention = kt.RandomSearch(
    build_advanced_attention_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=2,
    directory='tuner_advanced_attention',
    project_name='brain_age_ultra_attention',
    overwrite=True
)

tuner_attention.search(X_train_scaled, y_train, validation_split=0.2, epochs=500,
                         callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
                         verbose=1)
best_attention_model = tuner_attention.get_best_models(num_models=1)[0]


Trial 50 Complete [00h 07m 12s]
val_loss: 4.437074184417725

Best val_loss So Far: 4.287467002868652
Total elapsed time: 05h 18m 58s


d:\Brain Age\brain_age_env\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [10]:
#####################################
# Ultimate Ensemble Model: Stacking/Averaging the Predictions
#####################################
ensemble_input = Input(shape=(X_train_scaled.shape[1],))
out1 = best_residual_model(ensemble_input)
out2 = best_wide_deep_model(ensemble_input)
out3 = best_attention_model(ensemble_input)
ensemble_output = Average()([out1, out2, out3])
ensemble_model = Model(ensemble_input, ensemble_output, name="Ultimate_Ensemble_Model")

ensemble_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae', metrics=['mse','mae'])

# Fine-tune the ensemble on training data
ensemble_early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
ensemble_history = ensemble_model.fit(X_train_scaled, y_train,
                                      validation_split=0.2,
                                      epochs=500,
                                      batch_size=32,
                                      callbacks=[ensemble_early_stop],
                                      verbose=1)

Epoch 1/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 4.1790 - mae: 4.1247 - mse: 48.2026 - val_loss: 4.3849 - val_mae: 4.3311 - val_mse: 51.7757
Epoch 2/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 4.3190 - mae: 4.2653 - mse: 54.1329 - val_loss: 4.3637 - val_mae: 4.3099 - val_mse: 55.2245
Epoch 3/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.1015 - mae: 4.0478 - mse: 49.4096 - val_loss: 4.4143 - val_mae: 4.3605 - val_mse: 51.9728
Epoch 4/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.3075 - mae: 4.2537 - mse: 54.5660 - val_loss: 4.4291 - val_mae: 4.3751 - val_mse: 53.0795
Epoch 5/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4.2428 - mae: 4.1888 - mse: 53.1459 - val_loss: 4.4474 - val_mae: 4.3932 - val_mse: 53.0444
Epoch 6/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.1795 - mae: 4.1253 - mse: 50.8165 - val_loss: 4.6100 - val_mae: 4.5557 - val_mse: 57.2700
Epoch 7/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.2625 - mae: 4.2082 - mse: 

In [14]:
results = ensemble_model.evaluate(X_test_scaled, y_test, verbose=0)
for name, value in zip(ensemble_model.metrics_names, results):
    print(f"{name}: {value}")
    
predictions = ensemble_model.predict(X_test_scaled)

# Compute R² using scikit-learn
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print("r²:", r2)

# Save final ensemble model in the new Keras format
if os.path.exists("Ultimate_Brain_Model.keras"):
    os.remove("Ultimate_Brain_Model.keras")
ensemble_model.save("Ultimate_Brain_Model.keras")
print("Final ensemble model saved as Ultimate_Brain_Model.keras")

loss: 4.208929538726807
compile_metrics: 47.553428649902344
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
r²: 0.9638542679790925
Final ensemble model saved as Ultimate_Brain_Model.keras
